In [1]:
!pip install -qU langchain_community pypdf
!pip install langchain-groq
!pip install sentence_transformers
!pip install faiss-gpu
!pip install langgraph

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 

# Now put Your API key

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['GROQ_API_KEY'] = "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"

# Import the dependencies

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.retrievers import  ContextualCompressionRetriever
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain_core.documents import Document
from typing_extensions import TypedDict
from typing import List
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.load import dumps, loads
import uuid
import warnings
warnings.filterwarnings("ignore")

# Load the Documents

In [4]:
loader_1 = PyPDFLoader('/kaggle/input/kitchen-dataset/cook_book.pdf')
cook_pdf =loader_1.load()

In [5]:
cook_pdf

[Document(metadata={'source': '/kaggle/input/kitchen-dataset/cook_book.pdf', 'page': 0}, page_content='1. Nargisy Kababs: \n Ingredients \n\uf0b7 1 kg minced meat \n\uf0b7 2 medium onions, cut into large pieces \n\uf0b7 1/2 cup curd \n\uf0b7 1 tbsp ginger-garlic paste \n\uf0b7 1 tsp red chilli powder \n\uf0b7 1 tsp garam masala \n\uf0b7 1/4 tsp turmeric powder \n\uf0b7 1 tbsp soy sauce \n\uf0b7 salt as per taste \n\uf0b7 1 tbsp vinegar \n\uf0b7 1/4 cup \nFor Filling: \n\uf0b7 4 boiled eggs, finely chopped \n\uf0b7 1 cup coriander leaves, \n\uf0b7 finely chopped \n\uf0b7 1 cup mint leaves, finely chopped \n\uf0b7 2 or 3 green chilies finely chopped \n\uf0b7 1 tsp. chat masala \n For Frying: \n\uf0b7  1/4 cup besan \n\uf0b7  2 eggs \n\uf0b7  1/4 tsp salt \n\uf0b7  1/4 tsp turmeric powder \n \nMethod \nCook onions in oil until light brown. Add yogurt, garlic & ginger paste, salt, red chili, \ngaram masala, Soya sauce & vinegar to the onions and cook for 6 to 8 minutes. \n Add mince meat t

## Split the Documents

In [6]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
splits = text_spliter.split_documents(cook_pdf)

In [7]:
splits

[Document(metadata={'source': '/kaggle/input/kitchen-dataset/cook_book.pdf', 'page': 0}, page_content='1. Nargisy Kababs: \n Ingredients \n\uf0b7 1 kg minced meat \n\uf0b7 2 medium onions, cut into large pieces \n\uf0b7 1/2 cup curd \n\uf0b7 1 tbsp ginger-garlic paste \n\uf0b7 1 tsp red chilli powder \n\uf0b7 1 tsp garam masala \n\uf0b7 1/4 tsp turmeric powder \n\uf0b7 1 tbsp soy sauce \n\uf0b7 salt as per taste \n\uf0b7 1 tbsp vinegar \n\uf0b7 1/4 cup \nFor Filling: \n\uf0b7 4 boiled eggs, finely chopped \n\uf0b7 1 cup coriander leaves, \n\uf0b7 finely chopped \n\uf0b7 1 cup mint leaves, finely chopped \n\uf0b7 2 or 3 green chilies finely chopped \n\uf0b7 1 tsp. chat masala \n For Frying:'),
 Document(metadata={'source': '/kaggle/input/kitchen-dataset/cook_book.pdf', 'page': 0}, page_content='\uf0b7 finely chopped \n\uf0b7 1 cup mint leaves, finely chopped \n\uf0b7 2 or 3 green chilies finely chopped \n\uf0b7 1 tsp. chat masala \n For Frying: \n\uf0b7  1/4 cup besan \n\uf0b7  2 eggs \

## embedding the documents

In [8]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
vectorstore = FAISS.from_documents(documents=splits, embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# LLMs

## Retrieval Grader

In [9]:
# # I used the class for data modeling 
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

# LLM with function call
llm = ChatGroq(temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


In [10]:
# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGroq(temperature=0)
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

The context does not provide specific information about "agent memory". However, it does mention the process of marinating chicken, fish, and preparing stuffing for mince paste, as well as a recipe for Fish Vegetable Deluxe.


## Hallucination Grader

In [11]:
# I used the class for data modeling 
class HallucinationGrader(BaseModel):
    """Binary score for hallucination present in generation answer."""
    binary_score: str = Field(description= "Answer is grounded in the fact, 'yes' or 'No'")

# so here I use ChatGroq LLm
llm = ChatGroq(temperature=0)
structure_llm_grader = llm.with_structured_output(HallucinationGrader)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)


hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeDocuments(binary_score='no')

## Answer grader

In [12]:
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(description="Answer addresses the question, 'yes' or 'no'")
        
# LLM with function call
llm = ChatGroq(temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question,"generation": generation})

GradeAnswer(binary_score='no')

## Question re-writer

In [13]:
# LLM
llm = ChatGroq(temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying sematic intent / meaning. Just say the question. I dont need any explanation just question is enough"""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Here is the initial question: \n\n {question} \n Formulate an improved question."),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'"What is the concept of agent memory and how does it function in a vector database context?"'

# What is the difference *Retrival Grader*, "Hallucination Grader", "Answer Grader" and "Question Re-write".

## 1. **Relevance Grader (First Section)**

- **Purpose**: This part checks the relevance of a retrieved document based on a user's question. It is used to filter out irrelevant documents.
- **Data Model**: `GradeDocuments` contains a binary score (`yes` or `no`) to indicate whether the document is relevant to the user's question.
- **System Prompt**: The system instructs the grader to assess whether a document is relevant to the user's question, focusing on keywords and semantic meaning.
- **Example**: Given a document and a question (e.g., "agent memory"), the grader decides if the document is relevant.

## 2. **Hallucination Grader (Second Section)**

- **Purpose**: This section checks whether a generated answer is based on factual, retrieved documents to detect hallucinations (i.e., when the LLM makes up information).
- **Data Model**: `GradeHallucinations` uses a binary score (`yes` or `no`) to determine whether the generated answer is grounded in factual documents.
- **System Prompt**: The system asks the grader to determine whether the LLM's generation is supported by the facts.
- **Example**: Given a set of documents and an LLM's generation, it checks whether the generation is factually grounded.

## 3. **Answer Grader (Third Section)**

- **Purpose**: This section assesses whether the generated answer addresses the user's question directly.
- **Data Model**: `GradeAnswer` provides a binary score (`yes` or `no`) to check if the answer resolves the question.
- **System Prompt**: The system asks the grader to determine if the answer resolves the question.
- **Example**: It evaluates whether an LLM-generated answer directly answers the user's query.

## 4. **Question Re-writer (Fourth Section)**

- **Purpose**: This section focuses on improving the user question for better retrieval from a vector store, by rewriting the question to make it more semantically clear.
- **Data Model**: No data model like the previous sections. The LLM directly generates a better version of the question.
- **System Prompt**: The system instructs the LLM to rewrite the question for better retrieval performance, without providing an explanation.
- **Example**: It takes an initial user question and formulates a better one for retrieval purposes.

---

### Key Differences:

- **Tasks**: 
  - The first three sections (Relevance Grader, Hallucination Grader, Answer Grader) all evaluate specific aspects of the LLM's performance (relevance, hallucination, and answer quality).
  - The last section (Question Re-writer) rewrites the input question for better performance in document retrieval.

- **Data Models**:
  - The first three sections use specific `BaseModel` classes (`GradeDocuments`, `GradeHallucinations`, and `GradeAnswer`) to define the binary scoring mechanism, whereas the Question Re-writer does not require a custom data model.
  
- **Outputs**: 
  - The first three graders return binary outputs (`yes` or `no`), while the Question Re-writer returns an improved version of the original question.

- **LLM Function**: 
  - The LLM is used in all sections, but its role varies: evaluating relevance, factual grounding, and answers in the first three, while generating a rewritten question in the fourth.



# Grap State

In [14]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """
    question : str
    generation : str
    documents : List[str]

## Node:

In [15]:
# Rrtrieve the documents
def retrieve(state):
    """
    Retrieve documents
    
    Args:
        state (dict): The current graph state
        
    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}

In [16]:
# Generate the Answer
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

In [17]:
generation

'The context does not provide specific information about "agent memory". However, it does mention the process of marinating chicken, fish, and preparing stuffing for mince paste, as well as a recipe for Fish Vegetable Deluxe.'

In [18]:
# Grade the documents that it's relevant or not
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}

In [19]:
# This function will generate the question 
def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

## Edges:

In [20]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

In [21]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

# Build Graph

In [22]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae
workflow.add_node("transform_query", transform_query) # transform_query

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [23]:
from pprint import pprint

# Run
inputs = {"question": "How to cook the chicken?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('To cook the chicken, start by adding it to a pot with garlic and ginger on '
 'medium heat. Once the water dries, increase the heat, add remaining '
 'ingredients except green chilies and tomatoes, then cook covered for 5-10 '
 'minutes. You can also cook chicken with sliced tomato, onion, capsicum, '
 'garam masala powder, and salt in another pan. Ensure the chicken is tender '
 'before serving.')
